<a href="https://colab.research.google.com/github/yanipark/colab-project-name/blob/main/cifar10_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch #GPU 사용
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
np.random.seed(3)

ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (/usr/local/lib/python3.10/dist-packages/keras/api/preprocessing/image/__init__.py)

In [ ]:
from google.colab import drive
drive.mount ("/content/qdrive")


Mounted at /content/qdrive


In [ ]:
%cd /content/qdrive/MyDrive/archive

!unzip -qq "/content/qdrive/MyDrive/archive.zip"

/content/qdrive/MyDrive/archive
replace cifar10/labels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
train_datagen =ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.1)

train_set = train_datagen.flow_from_directory('/content/qdrive/MyDrive/archive/cifar10/train',
                                              target_size=(32,32),
                                              batch_size=50,
                                              class_mode='categorical',
                                              shuffle = True)

validation_set = train_datagen.flow_from_directory('/content/qdrive/MyDrive/archive/cifar10/train',
                                              target_size=(32,32),
                                              batch_size=50,
                                              class_mode='categorical',
                                              subset = 'validation',
                                              shuffle = True)

test_datagen = ImageDataGenerator(rescale=1./255)#test_set은 Normalization만 진행

test_set = test_datagen.flow_from_directory('/content/qdrive/MyDrive/archive/cifar10/test',
                                              target_size=(32,32),
                                              batch_size=50,
                                              class_mode='categorical',
                                              shuffle= False)




Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Flatten, Dense, Dropout,BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers.legacy import Adam
from keras.utils import plot_model

In [ ]:
#model(Dense, Dropout, BatchNormalization)이용 추가로 flatten
model= Sequential()

model.add(Conv2D(32,(3,3), input_shape = (32,32,3)))
model.add(BatchNormalization())
model.add(Activation('relu')) #활성화 함수

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) #4배 축소

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten()) #활성화함수를 사용하기전에 1차원으로 바꿔줌

#  특징 추출 -> Classification
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.5)) # 과적합을 피하기 위해서 dropout 사용. 50% 노드를 사용
model.add(Dense(10)) #최종출력 노드의 수는 클래스 갯수와 같은 10개로
model.add(Activation('softmax')) #출력층(다중 클래스 분류 문제) = softmax function
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 30, 30, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 30, 30, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 64)       256       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 28, 28, 64)        0

In [ ]:
#compile
model.compile(optimizer="Adam",
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 30, 30, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 30, 30, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 64)       256       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 28, 28, 64)        0

In [ ]:
#fit(model 학습)
model.fit_generator(train_set,
                    steps_per_epoch=len(train_set),
                    epochs=30,
                    validation_data=test_set,
                    validation_steps = len(test_set))

<ipython-input-9-4875879e26f9>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_set,


Epoch 1/30
1000/1000 [==============================] - 152s 152ms/step - loss: 2.0051 - accuracy: 0.2420 - val_loss: 1.6219 - val_accuracy: 0.4066
Epoch 2/30
1000/1000 [==============================] - 138s 138ms/step - loss: 1.7179 - accuracy: 0.3695 - val_loss: 1.5128 - val_accuracy: 0.4270
Epoch 3/30
1000/1000 [==============================] - 138s 138ms/step - loss: 1.5910 - accuracy: 0.4223 - val_loss: 1.4020 - val_accuracy: 0.4948
Epoch 4/30
1000/1000 [==============================] - 142s 142ms/step - loss: 1.5043 - accuracy: 0.4635 - val_loss: 1.4303 - val_accuracy: 0.4870
Epoch 5/30
1000/1000 [==============================] - 138s 138ms/step - loss: 1.4396 - accuracy: 0.4915 - val_loss: 1.2580 - val_accuracy: 0.5520
Epoch 6/30
1000/1000 [==============================] - 139s 139ms/step - loss: 1.3763 - accuracy: 0.5196 - val_loss: 1.3101 - val_accuracy: 0.5401
Epoch 7/30
1000/1000 [==============================] - 134s 134ms/step - loss: 1.3354 - accuracy: 0.5399 - val_

In [ ]:
#모델 성능 테스트 데이터로 평가
test_hist = model.evaluate(test_set)

NameError: ignored

In [ ]:
#acc&loss 그래프 그리기
 # plot loss
from tensorflow.keras.utils import plot_model
plot_model(model, to_file = 'model.png')

NameError: ignored